In [1]:
base = "/data_nfs/"

In [2]:
import torch as t
#from pytorch_grad_cam import GradCAM
#from pytorch_grad_cam.utils.image import show_cam_on_image
#from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import cv2 
import sys
import os
from sklearn.mixture import GaussianMixture
import seaborn as sns

sys.path.append(os.path.join(base, "je30bery/melanoma_data/MAGICAL/model/"))
from data import MelanomaData
import numpy as np
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
import pickle
import pandas as pd
plt.style.use('dark_background')
os.environ["CUDA_VISIBLE_DEVICES"]="1"
sys.path.append("../../MAGICAL/data_utils")
from data_utils import *

In [3]:
markers = ['ADAM10', 'Bcl-2', 'CD10', 'CD107a', 'CD13', 'CD138', 'CD14', 'CD1a', 'CD2', 'CD25', 'CD271', 'CD3', 'CD36', 'CD4', 'CD44', 'CD45', 'CD45RA', 'CD45RO', 'CD5', 'CD56', 'CD6', 'CD63', 'CD66abce', 'CD7', 'CD71', 'CD8', 'CD9', 'CD95', 'Collagen IV', 'Cytokeratin-14', 'EBF-P', 'EGFR', 'EGFR-AF488', 'HLA-ABC', 'HLA-DR', 'KIP1', 'Ki67', 'L302', 'MCSP', 'Melan-A', 'Nestin-AF488', 'Notch-1', 'Notch-3', 'PPARgamma', 'PPB', 'RIM3', 'TAP73', 'Vimentin', 'p63', 'phospho-Connexin']    
#from model import EfficientnetWithFinetuning
model = t.load("/data_nfs/je30bery/melanoma_data/model/model_2024-01-29 17:37:18.558634.pt", map_location="cuda:0")

#model = EfficientnetWithFinetuning(indim=len(markers))
#model.load_state_dict(t.load("/data_nfs/je30bery/melanoma_data/model/training/saved_models/model_2023-11-20 16:27:56.558630_f1=0.9260606060606061_acc=0.9166666666666666_11.pt"))

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
sys.path.append("../../MAGICAL/smoothgrad")

from individual_channel_gradcam import GradCAM

In [6]:
use_rois = False
rois = [os.path.splitext(f)[0][7:] for f in os.listdir(os.path.join(base, "je30bery/melanoma_data/ROI/filled")) if "filled" in f]

In [7]:
data = get_data_csv(base)
data = data.reset_index()

In [8]:
#changed = ["Melanoma_16_201908011518_4",
#"Melanoma_17_201909111532_4",
#"Melanoma_22_202003091404_4",
#"Melanoma_24_202004021245_3",
#"Melanoma_36_202009111023_3",
#"Melanoma_39_202010141128_1",
#"Nevi_10_201811131523_4",
#"Nevi_16_201910151344_4"]
#data = data[data["file_path"].isin(changed)]

In [9]:
def get_binary(v, seg_file):
    v = v / np.max(v) * 255
    binary = cv2.resize(v.astype(np.uint8), (2018, 2018))
    _, binary = cv2.threshold(binary, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    roi_segments = np.unique((binary > 0) * seg_file)
    roi_seg_file = (np.where(np.isin(seg_file, roi_segments), seg_file, 0) > 0) * 255
    roi_seg_file = cv2.resize(roi_seg_file.astype(np.uint8), (512, 512))
    return roi_seg_file / 255., roi_segments

In [10]:
model = model.cuda()

In [11]:
if use_rois:
    vis = np.load("visualizations_regressor.npy", allow_pickle=True).item()
else:
    model.eval()    
    vis = dict()
    for layer in [5]:
        vis[layer] = dict()
        print(layer)
        GradC = GradCAM(model=model, target_layer=layer, mode="in")
        dl = t.utils.data.DataLoader(MelanomaData(markers, dataset="Melanoma", data=data, mode="val"), batch_size=1, shuffle=False)
        it = iter(dl)
        visualizations = list()
        d = 0
        while True:
            try:
                fov = data.iloc[d]["file_path"] 
                path = os.path.join(base, f"je30bery/melanoma_data/MAGICAL/data/ROIs/regression_model/{layer}/")
                os.makedirs(path, exist_ok=True)
                path = os.path.join(path, fov + "_binary.npy")
                #if os.path.exists(path):
                #    vis[layer][d] = np.load(path)
                #    print("skipping", fov)
                #    if layer != 5:
                #        continue
                input_t, label = next(it)
                
            except:
                break
 
            input_t.requires_grad = True
            segmented = os.path.join(base, "datasets/melc/melanoma/", "segmented", f'{fov}_cells.npy')
            with open(segmented, "rb") as openfile:
                seg_file = np.load(openfile)
            
            gc = GradC.cam(input_t.cuda(), n_smooth=10)  
            if layer == 0:
                gc = gc * input_t.detach().cpu().numpy()
                gc *= (gc > 0)
            gc = np.mean(gc, axis=(0,1))
            binary, roi_segments = get_binary(gc, seg_file)
            vis[layer][d] = binary
            d += 1

            path = os.path.join(base, f"je30bery/melanoma_data/MAGICAL/data/ROIs/regression_model/{layer}/")
            np.save(os.path.join(path, fov + "_binary.npy"), binary)
            with open(os.path.join(path, fov + "_idxs.pkl"), "wb") as fp:   
                pickle.dump(roi_segments, fp)

5


In [12]:
datasdf

NameError: name 'datasdf' is not defined

In [ ]:
for d in range(len(data)):
    fov = data.iloc[d]["file_path"]
    print(d, fov)
    for combination in [range(4), range(5), range(6)]:
        v_combined = np.ndarray((len(combination), 2018, 2018))
        for j in range(len(combination)):
            v_combined[j] = cv2.resize(vis[j][d], (2018, 2018))
        v = v_combined.mean(axis=0)
        binary, roi_segments = get_binary(v, seg_file)
        path = os.path.join(base, f"je30bery/melanoma_data/MAGICAL/data/ROIs/classification_model/combined_0-{len(combination)-1}/")
        path = os.path.join(path, fov + "_binary.npy")
        np.save(path, binary)
        with open(os.path.join(os.path.join(base, f"je30bery/melanoma_data/MAGICAL/data/ROIs/classification_model/combined_0-{len(combination)-1}/"), fov + "_idxs.pkl"), "wb") as fp:   
            pickle.dump(roi_segments, fp)
            print("dumped", fov)

In [ ]:
#dl = t.utils.data.DataLoader(MelanomaData(markers, dataset, data, mode="val"), batch_size=1, shuffle=False)
#it = iter(dl)
plot = False
use_rois = True

ious = pd.DataFrame()
if 1:
    for d in range(len(data)):
        fov = data.iloc[d]["file_path"]
        segmented = os.path.join(base, "datasets/melc/melanoma/", "segmented", f'{fov}_cells.npy')
        with open(segmented, "rb") as openfile:
            seg_file = np.load(openfile)
        
        if use_rois:
            roi = cv2.imread(os.path.join(base, f"je30bery/melanoma_data/ROI/filled/filled_{fov}.tif"), cv2.IMREAD_GRAYSCALE)
            roi = cv2.resize(roi, (512, 512))
            if d == 5:
                roi = (roi > 15).astype(float)
            else:
                roi = (roi > 0).astype(float)
        
        bins = dict()
        i = 0
        for layer in range(9):
            if layer < 6:
                v = vis[layer][d]
                binary = get_binary(v, seg_file)
                
            else:
                v_combined = np.ndarray((layer - 2, 2018, 2018))
                for j in range(layer - 2):
                    v_combined[j] = cv2.resize(vis[j][d], (2018, 2018))
                v = v_combined.mean(axis=0)
                binary = get_binary(v, seg_file)

            bins[i] = binary
            i += 1
            path = os.path.join(base, f"je30bery/melanoma_data/model/GradCam/ROIs/{layer}/")
            os.makedirs(path, exist_ok=True)
            path = os.path.join(path, fov + "binary.npy")
            np.save(path, binary)
            if use_rois:
                ious.loc[d, layer] = intersection_over_union(binary, roi)
        """
        for i in range(9):
            for j in range(9):
                if j <= i:
                    continue
                print(i, j)
                combined_vis = vis[i][0][d] + vis[j][0][d] 
                binary = combined_vis > get_cutoff(combined_vis)
                binary = bins[i] * bins[j]
                ious.loc[d, f"{i}*{j}"] = intersection_over_union(binary, roi)
        """
        if plot:
            f, axs = plt.subplots(3, 3, figsize=(10, 10))
            i = 0
            for layer in range(9):
                
                if i >= 6:
                    rgb = np.array([np.zeros_like(bins[i]), bins[i], roi]).transpose((1, 2, 0))
                else:
                    rgb = np.array([bins[i], np.zeros_like(bins[i]), roi]).transpose((1, 2, 0))
                axs[i // 3, i % 3].imshow(rgb)
                axs[i // 3, i % 3].axis("off")
                i += 1
            plt.show()
            _ = input("Proceed")
            
            clear_output(wait=True)
            

In [ ]:
#layer = 7
data = data#data = data[data["file_path"].isin(changed)]
f, axs = plt.subplots(4,4, figsize=(15,15))
for d in range(len(data)):
    fov = data.iloc[d]["file_path"]
    segmented = os.path.join(base, "datasets/melc/melanoma/", "segmented", f'{fov}_cells.npy')
    with open(segmented, "rb") as openfile:
        seg_file = np.load(openfile)
     
    roi = cv2.imread(os.path.join(base, f"je30bery/melanoma_data/ROI/filled/filled_{fov}.tif"), cv2.IMREAD_GRAYSCALE)
    roi = cv2.resize(roi, (512, 512))
    if d == 5:
        roi = (roi > 15).astype(float)
    else:
        roi = (roi > 0).astype(float)
    #v = vis[layer][d]
    v_combined = np.ndarray((4, 2018, 2018))
    for j in range(4):
        v_combined[j] = cv2.resize(vis[j][d], (2018, 2018))
    v = v_combined.mean(axis=0)
    binary = get_binary(v, seg_file)
    binary = get_binary(v, seg_file)
    rgb = np.array([binary, np.zeros_like(binary), roi]).transpose((1, 2, 0))
    axs[d // 4, d % 4].imshow(rgb)
    axs[d // 4, d % 4].axis("off")
    axs[d // 4, d % 4].set_title(fov)
plt.show()

In [ ]:
def get_gaussian(x, mu, sigma):
    return 1/(np.sqrt(2 * np.pi) * sigma) * np.exp(-0.5 * (x - mu)**2 / sigma**2)
    
def get_cutoff(visualization):
    gm_bimodal = GaussianMixture(n_components=2).fit(visualization.flatten()[np.newaxis].T)
    x = np.linspace(0, 1, 10000)
    y1 = get_gaussian(x, gm_bimodal.means_[0], gm_bimodal.covariances_[0, 0, 0])
    y2 = get_gaussian(x, gm_bimodal.means_[1], gm_bimodal.covariances_[1, 0, 0])
    intersect = x[np.argmin(np.abs(y1-y2))]
    return th2, intersect

In [ ]:
def intersection_over_union(a, b):
    intersection = np.sum(np.logical_and(a, b))
    union = np.sum(np.logical_or(a, b))
    return intersection/union

In [ ]:
plt.imshow(seg_file)

In [ ]:
plt.figure(figsize=(20, 5))
plt.xticks(rotation=90)
sns.boxplot(ious)

In [ ]:
ious.mean(axis=0)

In [ ]:
ious

In [ ]:
plt.figure(figsize=(20, 5))
plt.xticks(rotation=90)
sns.boxplot(ious)

In [ ]:
ious.loc["mean"] = ious.mean()
# thresh 10, layer 4
# thresh 1, layer 3
# thresh 10, layer 6
# thresh 10, layer 8
# thresh 1, layer 2

In [ ]:
plt.imshow(binary)

In [ ]:
ious.sort_values("mean", axis=1, ascending=False)

In [ ]:
if 0:
    f, axs = plt.subplots(3, 6, figsize=(18,9))
    for i, k in enumerate(path_dict):
        boundaries = cv2.resize((cv2.imread(path_dict[k], cv2.IMREAD_GRAYSCALE) > 0).astype(np.uint8), (512,512))
        axs[int(i/6), i%6].imshow(mark_boundaries(visualizations[k], boundaries, color=(1,0,0)))
        axs[int(i/6), i%6].axis("off")
        axs[int(i/6), i%6].set_title(k)
    plt.suptitle("GradCAM results with marked expert annotations")
    plt.tight_layout()
    plt.show()
    #plt.savefig("gradcam_val.pdf")

In [ ]:
if 0:
    h, w = 13, 3
    f, axs = plt.subplots(h, w, figsize=(15, 50))
    for idx in range(len(vdata)):
        axs[idx % h, int(idx / h)].imshow(visualizations[idx])
        title = os.path.basename(vdata[idx]) #"Melanoma" if labels[idx] == 0 else "Nevi"
        axs[idx % h, int(idx / h)].set_title(title)
        axs[idx % h, int(idx / h)].axis("off")
    plt.savefig("gradcam_val.pdf")
    plt.show()

In [ ]:
grayscale_cam.shape